In [70]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Model

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True,threshold=np.inf)
pd.options.display.max_rows = 999

In [71]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

df = pd.read_csv("kddcup99_csv.csv")
col2encode = ["protocol_type","service","flag"]
ohe_col = pd.get_dummies(df[col2encode],columns=col2encode)
df = df.drop(columns=col2encode)
df_ohe = pd.concat([df,ohe_col],axis=1)

attack_values = pd.unique(df_ohe["label"].values.ravel())
attack_values = attack_values[1:] ## Remove "normal" as it is not an attack

### Create a dict to remap attack to True and normal to False
d_attack = {"normal":False}
for a in attack_values:
    d_attack.update({a:True})

label = df_ohe["label"].map(d_attack)
df_ohe = df_ohe.drop(columns="label",axis=1)

X_train, X_test, y_train, y_test = train_test_split(df_ohe, label, test_size=0.25, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_sc_train = scaler.transform(X_train)
X_sc_test = scaler.transform(X_test)

NCOMPONENTS = 36

pca = PCA(n_components=NCOMPONENTS)
X_pca_train = pca.fit_transform(X_sc_train)
X_pca_test = pca.transform(X_sc_test)

print(X_pca_train.shape)
print(X_pca_test.shape)

X_pca_train  = X_pca_train.reshape(370515,6,6)
X_pca_test  = X_pca_test.reshape(123505,6,6)

print(X_pca_train.shape)
print(X_pca_test.shape)


(370515, 36)
(123505, 36)
(370515, 6, 6)
(123505, 6, 6)


In [72]:

from keras import Input
from keras.regularizers import l1
from keras.layers import MaxPooling2D, Flatten, BatchNormalization, Dense
from qkeras import QConv2DBatchnorm, QActivation, QDense

x = x_in = Input(shape=(6,6,1))

x = QConv2DBatchnorm(8, kernel_size=(3,3), strides=(1,1),
                     kernel_quantizer="quantized_bits(6,0,alpha=1)",
                     bias_quantizer="quantized_bits(6,0,alpha=1)",
                     kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=True,
                     name='fused_convbn')(x)
x = QActivation('quantized_relu(6)',name='conv_act1')(x)
x = MaxPooling2D(pool_size = (2,2),name='pool')(x)
x = Flatten()(x)
x = BatchNormalization(name='bn_dense')(x)
x_out = Dense(1,
           name='output_dense')(x)
#x_out = Activation('softmax',name='output_softmax')(x)
model = Model(inputs=[x_in], outputs=[x_out], name='qkeras')

model.summary()

Model: "qkeras"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 6, 6, 1)]         0         
                                                                 
 fused_convbn (QConv2DBatchn  (None, 4, 4, 8)          113       
 orm)                                                            
                                                                 
 conv_act1 (QActivation)     (None, 4, 4, 8)           0         
                                                                 
 pool (MaxPooling2D)         (None, 2, 2, 8)           0         
                                                                 
 flatten_10 (Flatten)        (None, 32)                0         
                                                                 
 bn_dense (BatchNormalizatio  (None, 32)               128       
 n)                                                         

In [73]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics =['accuracy']
)

model.fit(X_pca_train, y_train, batch_size=64, epochs=10,validation_data=(X_pca_test,y_test))

Epoch 1/10
5790/5790 [==============================] - 9s 1ms/step - loss: 2.4078 - accuracy: 0.8370 - val_loss: 0.8085 - val_accuracy: 0.9455

In [74]:
print("Evaluate on test data")
results = model.evaluate(X_pca_test, y_test)
print("test acc:", results)
model.save("model3")

Evaluate on test data
3860/3860 [==============================] - 3s 653us/step - loss: 0.0704 - accuracy: 0.9932

INFO:tensorflow:Assets written to: model3\assets


INFO:tensorflow:Assets written to: model3\assets
